# Big Data Project H600 / L-Group 
## Real world Data Exploration, Integration, Cleasning, Transformation and Analysis
### Can be run on a personnal computer

 **Note: this notebook assumes that:**
    - The sampled data is in "MY_PARENT_FOLDER/data/sampled/" folder.
    - You can run the bash script "download_metadata.sh" to download data and metadata in the correct folders to execute the jupyter notebooks.

Data:
The New York City Taxi and Limousine Commission (or TLC for short) has been publishing
records about taxi trips in New York since 2009. 

The TLC trip dataset actually consists of 4 sub-datasets:

    1.Yellow taxi records are records that record trip information of New York's famous yellow taxi cars

    2.Green taxi records are records that record trip information by so-called 'boro' taxis, a newer service introduced in August of 2013 to improve taxi service and availability in the boroughs. 

    3.FHV records (short for 'For Hire Vehicles') record information from services that offered for-hire vehicles (such as Uber, Lyft, Via, and Juno), but also luxury limousine bases.

    4.High volume FHV (FHVHV for short) are FHV records offered by services that make more than 10,000 trips per day

In [2]:
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=3g  pyspark-shell"
from pyspark.sql import SparkSession
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass

# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("H600 L-Group") \
    .getOrCreate()
    
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext

#in local mode, you will be able to access the Spark GUI at http://localhost:4040



Spark application already started. Terminating existing application and starting new one


In [10]:
# Check if the files and forlder are on the good folder to exectue the following code.
!./check_files.sh


------------ READY TO START -----------


### Check for the files shema
The schema of the files have changes over time (name, order and numbers of column). The following script look for the kind of change and the date when it occurs.

In [84]:
import os
import glob
import pandas as pd
import math
import numpy as np

# Get the list of file
taxi_brand="green"
list_files = {}
list_files[taxi_brand] = []
nb_files = 0
size = []
# List the file from the same taxi company brand 
for file in glob.glob("data/sampled/%s*.csv" %(taxi_brand)):
    nb_files = nb_files+1
    # Save in list the file name
    list_files[taxi_brand].append(file)
    "size.append(os.path.getsize(file))
# Order the files list by file names
print(np.sum(size))
list_files[taxi_brand].sort()
print("- For the taxi brand called %s there are %i files." %(taxi_brand, nb_files))
for yr in range(0,nb_files):
    # Read file by file
    df = pd.read_csv(list_files[taxi_brand][yr],sep=',')
    # Extract the head of the file
    head = list(df)
    # Save first file schema as reference
    if yr == 0 :
        nb_col_ref = len(head)
        col_name_ref = [ x.lower() for x in head ]
        # print(col_name_ref)
    else :
        # Compare reference schema with the shema of all the others files
        if len(head) != nb_col_ref :
            # If the numbers of column is different from reference yr check what is the new column name
            print("     there is diff in %i - %i :" %(int(list_files[taxi_brand][yr][28:32]),int(list_files[taxi_brand][yr][33:35])))
            diff_nb_col = len(head) - nb_col_ref
            ## print(len(head),nb_col_ref)
            # lower case for all col names
            head_lower_new = [ x.lower() for x in head ]
            # find the new/remove col name
            diff_name_col = [ x for x in head_lower_new if x not in col_name_ref ]
            col_name_ref = head_lower_new
            nb_col_ref = len(head_lower_new)
            # Check if the order of the column have changed
            pos_col_change = []
            for i in range(0,len(diff_name_col)) :
                pos_col_change.append(head_lower_new.index(diff_name_col[i]))
            if diff_nb_col > 0 :
                print("          %i col add: %s in position %s" %(diff_nb_col, ' and '.join(diff_name_col), str(pos_col_change)))
            else :
                print("          %i col remove: %s in position %s" %(abs(diff_nb_col), ' and '.join(diff_name_col), str(pos_col_change)))
            
        else :    
    


#Check the files
    
    # Check if the file name have all the same lenght to determine start and end date
#    if nb_files == 1:
#        len_filename_ref = len(file)
#    if len(file) != len_filename_ref:
 #       print("The file %s in pos %i have longer file name of %i " %(file, nb_files, (len(file)-len_filename_ref)))

        
        
        
#print("The file name pattern look likes %s" %(file) )
#yr_file=[]
#mth_file=[]
#for yr in range(0,nb_files):
#    tmp1 = int(list_files[taxi_brand][yr][28:32])
#    print(tmp1)
#    int(list_files[taxi_brand][yr][33:35])
#    df = pd.DataFrame({'year': int(list_files[taxi_brand][yr][28:32]),
#                       'month': int(list_files[taxi_brand][yr][33:35])})
#    pd.to_datetime(df[["year","month"]])

#df = pd.DataFrame({'year'} : )

19945234
- For the taxi brand called green there are 76 files.
     there is diff in 2015 - 1 :
          1 col add: improvement_surcharge in position [17]
     there is diff in 2016 - 7 :
          2 col remove: pulocationid and dolocationid in position [5, 6]
     there is diff in 2019 - 1 :
          1 col add: congestion_surcharge in position [19]


In [10]:
# Load the contents of a file into an RDD. Note - when run on the cluster this load from HDFS (inside /user/$USER/)
# if you really want to load from HDFS, you can also put the full HDFS url, e.g.
# hdfs://public00:8020/user/<your_user_id_here>/data/books/pg20417.txt
fileName = 'Data/green_tripdata_2020-01.csv'
TaxiRDD = sc.textFile(fileName)

In [11]:
TaxiRDD.take(5)

['VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge',
 ',2020-01-27 06:40:00,2020-01-27 07:25:00,,,159,61,,19.84,40.95,2.75,0,0,6.12,,0.3,50.12,,,',
 '2,2020-01-18 01:01:42,2020-01-18 01:04:57,N,1,7,146,1,.56,4.5,0.5,0.5,1.45,0,,0.3,7.25,1,1,0',
 '2,2020-01-15 13:52:01,2020-01-15 14:13:09,N,1,134,77,1,5.87,22,0,0.5,0,0,,0.3,22.8,1,1,0',
 '2,2020-01-30 23:49:37,2020-01-31 00:12:17,N,1,42,143,1,5.40,20,0.5,0.5,4.81,0,,0.3,28.86,1,1,2.75']

In [13]:
tupleRDD = TaxiRDD.map(lambda line: line.split())
tupleRDD.take(3)

[['VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge'],
 [',2020-01-27',
  '06:40:00,2020-01-27',
  '07:25:00,,,159,61,,19.84,40.95,2.75,0,0,6.12,,0.3,50.12,,,'],
 ['2,2020-01-18',
  '01:01:42,2020-01-18',
  '01:04:57,N,1,7,146,1,.56,4.5,0.5,0.5,1.45,0,,0.3,7.25,1,1,0']]

In [14]:
wordsRDD = TaxiRDD.flatMap(lambda line: line.split())
wordsRDD.take(5)

['VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge',
 ',2020-01-27',
 '06:40:00,2020-01-27',
 '07:25:00,,,159,61,,19.84,40.95,2.75,0,0,6.12,,0.3,50.12,,,',
 '2,2020-01-18']

In [15]:
wordsRDD.count()

2686

In [16]:
wordsRDD.map(lambda x: 1).reduce(lambda a,b: a+b)

2686

### Do a word count on all files

In [17]:
allTaxiRDD = sc.textFile('Data/*.csv')

In [19]:
allTaxiRDD.count()

5036382

In [20]:
allTaxiRDD.count()

5036382

### Yellow taxi records